# Backtes the three PTS Methods

In [1]:
from Methods.DmHelpers import *
from Methods.CointHelpers import *
from Methods.CopulasHelpers import *

from Helpers.DataHelpers import *
from Helpers.PlotHelpers import *
from Helpers.ModuleHelpers import *
from Helpers.BacktestHelpers import *

In [2]:
# Donwnload data
tickers = pd.read_excel('Data/SandP500list.xlsx')
tickers = tickers[tickers['sector'] == 'Financials']
names = tickers['symbol'].tolist()

START_DATE = pd.to_datetime('2020-01-01') # Every year takes approximately 30 seconds for coint and dm
END_DATE = pd.to_datetime('2024-07-01') # test 6 months

data = download_data(names, START_DATE, END_DATE)

[*********************100%***********************]  65 of 65 completed


5 Failed downloads:
['WLTW', 'FRC', 'RE', 'BRK.B', 'PBCT']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [3]:
import pandas as pd

# Assuming END_DATE, START_DATE, train_test_dates, train_test_split, dm_get_signals_backtest, coint_get_signals_backtest, copula_get_signals_backtest, and print_backtest_info are properly defined

num_testing_periods = pd.to_datetime(END_DATE) - pd.to_datetime(START_DATE)
start_date, end_date = START_DATE, START_DATE + pd.DateOffset(months=18)

dm_backtest_results = pd.DataFrame()
coint_backtest_results = pd.DataFrame()
copula_backtest_results = pd.DataFrame()

counter = 0

while end_date < END_DATE:
    # Split data into training and testing
    train_start, train_end, test_start, test_end = train_test_dates(start_date)
    train_data, test_data = train_test_split(data, train_start, train_end, test_start, test_end)
    
    # Run backtests for each strategy
    dm_signals, dm_pairs = dm_get_signals_backtest(train_data, test_data, threshold=1)
    coint_signals, coint_pairs = coint_get_signals_backtest(train_data, test_data, threshold=1)
    copula_signals, copula_pairs = copula_get_signals_backtest(train_data, test_data) #! Threshold

    # Helper function to process each set of signals
    def process_signals(signals, results_df, label):
        tuples = [(label, col) for col in signals.columns]
        multi_index = pd.MultiIndex.from_tuples(tuples, names=['Period', 'Signal'])
        period_signals = pd.DataFrame(signals.values, index=signals.index, columns=multi_index)
        return pd.concat([results_df, period_signals], axis=1)

    # Format the period label for the big column
    period_label = f"{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}"

    # Process each method's signals and update the respective DataFrame
    dm_backtest_results = process_signals(dm_signals, dm_backtest_results, period_label)
    coint_backtest_results = process_signals(coint_signals, coint_backtest_results, period_label)
    copula_backtest_results = process_signals(copula_signals, copula_backtest_results, period_label)

    # Update for next period
    print_backtest_info(counter, start_date, end_date)
    start_date, end_date = update_dates(start_date)
    counter += 1


Backtest Period: 1
Start Date: 2020-01-01 End Date: 2021-07-01
-------------------------------------------
Backtest Period: 2
Start Date: 2020-02-01 End Date: 2021-08-01
-------------------------------------------


KeyboardInterrupt: 